In [1]:
import requests
from bs4 import BeautifulSoup,SoupStrainer
import pandas as pd
from IPython.display import Markdown, display,HTML,Image
from selenium import webdriver
import time
from pandas.plotting import table 
import matplotlib.pyplot as plt
import string

# Borrowed the offset idea from this (much more professionally done) tableau page
# https://public.tableau.com/app/profile/heidi.stockton/viz/PuzzlesofJaneStreet/JaneStreet
# No adjustments for similar names etc (apologies)

In [2]:
# get the solution links
pages = ['https://www.janestreet.com/puzzles/archive/index.html']+['https://www.janestreet.com/puzzles/archive/page'+str(i)+'/index.html' for i in range(2,13)]
links =  ['https://www.janestreet.com/puzzles/current-puzzle/']
for url in pages:
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'html.parser')
    links += ['https://www.janestreet.com/'+i['href'] for i in soup.find_all('a', {'class' :'solution-link'})]   

In [3]:
# scrape the names of solvers
participants = []
order = 1
driver = webdriver.Firefox()
for url in links:
    driver.get(url)
    time.sleep(5)
    htmlSource = driver.page_source
    soup = BeautifulSoup(htmlSource, 'html.parser')
    participants += [[str(url).split("/")[-1],i.split(" (")[0],order] for i in soup.find_all('p', {'class' :'correct-submissions'})[0].stripped_strings]
    order +=1

In [4]:
outputs = []
for newcomer_offset in [6]:
    df = pd.DataFrame(participants,columns = ['puzzle','Name','Order'])
    df['Name']=df['Name'].str.lstrip("1234567890. ")
    df['Name'] = df['Name'].str.title()
    df2 = df[['Name','Order']].groupby('Name').agg({'Name':['count'],'Order': ['max']})
    df2.columns = [' '.join(col).strip() for col in df2.columns.values]
    df2.columns = ['Solved','Attempts']
    df2['Score']=df2['Solved']/(df2['Attempts']+ newcomer_offset)
   
df2.sort_values(['Score','Solved'],ascending=False)[:15]

,Solved,Attempts,Score
Name,,,
Evan Semet,36,36,0.857143
Karl Mahlburg,54,60,0.818182
Senthil Rajasekaran,60,71,0.779221
Gareth Owen,42,51,0.736842
Aaditya Raghavan,44,54,0.733333
Blaine Hill,16,16,0.727273
Lazar Ilic,39,48,0.722222
Danica Xiong,17,18,0.708333
Calvin Pozderac,63,84,0.700000


In [5]:
df2.sort_values('Solved',ascending=False)[:15]

,Solved,Attempts,Score
Name,,,
Calvin Pozderac,63,84,0.700000
Senthil Rajasekaran,60,71,0.779221
Karl Mahlburg,54,60,0.818182
Sean Egan,52,78,0.619048
Keith Schneider,48,75,0.592593
Aaditya Raghavan,44,54,0.733333
Gareth Owen,42,51,0.736842
Heidi Stockton,40,97,0.388350
Lazar Ilic,39,48,0.722222


In [6]:
import xlwings as xw
sht = xw.Book().sheets[0]
sht.range('a1').options(index=False).value =  df

In [7]:
links

['https://www.janestreet.com/puzzles/current-puzzle/',
 'https://www.janestreet.com//puzzles/fences-2-solution',
 'https://www.janestreet.com//puzzles/tree-edge-triage-solution',
 'https://www.janestreet.com//puzzles/many-happy-returns-solution',
 'https://www.janestreet.com//puzzles/altered-states-2-solution',
 'https://www.janestreet.com//puzzles/number-cross-4-solution',
 'https://www.janestreet.com//puzzles/robot-capture-the-flag-solution',
 'https://www.janestreet.com//puzzles/hooks-10-solution',
 'https://www.janestreet.com//puzzles/some-off-square-solution',
 'https://www.janestreet.com//puzzles/some-f-squares-solution',
 'https://www.janestreet.com//puzzles/hall-of-mirrors-2-solution',
 'https://www.janestreet.com//puzzles/knight-moves-5-solution',
 'https://www.janestreet.com//puzzles/a-weird-tour-solution',
 'https://www.janestreet.com//puzzles/getting-from-a-to-b-solution',
 'https://www.janestreet.com//puzzles/single-cross-2-solution',
 'https://www.janestreet.com//puzzles/